# Evaluación Automatizada del Chatbot Turístico

Este notebook ejecuta automáticamente una serie de preguntas predefinidas a través del chatbot y guarda las respuestas para su análisis posterior.

In [1]:
# Importar bibliotecas necesarias
import os
import sys
import json
import pandas as pd
from dotenv import load_dotenv

# Configurar el path para encontrar los módulos
current_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.abspath(os.path.join(current_dir, '..'))
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)
print(f"Directorio src agregado al path: {src_dir}")

# Cargar variables de entorno
env_path = os.path.join(os.path.dirname(src_dir), '.env')
load_dotenv(env_path)
print(f"Archivo .env cargado desde: {env_path}")

Directorio src agregado al path: c:\Users\Janet\Documents\GitHub\GPTur\src
Archivo .env cargado desde: c:\Users\Janet\Documents\GitHub\GPTur\.env


In [2]:
# Verificar la configuración del entorno
def check_environment():
    checks = {
        "Python Path": src_dir in sys.path,
        "Módulo agents": False,
        "Módulo chatbot": False,
        "Módulo vector_db": False,
        ".env file": os.path.exists(env_path),
        "questions.csv": os.path.exists('questions.csv')
    }
    
    try:
        import agents
        checks["Módulo agents"] = True
    except ImportError as e:
        print(f"Error importando agents: {e}")
        
    try:
        import chatbot
        checks["Módulo chatbot"] = True
    except ImportError as e:
        print(f"Error importando chatbot: {e}")
        
    try:
        import vector_db
        checks["Módulo vector_db"] = True
    except ImportError as e:
        print(f"Error importando vector_db: {e}")
    
    print("\n=== Verificación del Entorno ===")
    all_passed = True
    for check, passed in checks.items():
        status = "✓" if passed else "✗"
        if not passed:
            all_passed = False
        print(f"{status} {check}")
    
    if not all_passed:
        print("\n⚠ Hay problemas en la configuración que deben ser resueltos")
    else:
        print("\n✓ Todo está correctamente configurado")
    
    return all_passed

# Ejecutar verificación
environment_ok = check_environment()


=== Verificación del Entorno ===
✓ Python Path
✓ Módulo agents
✓ Módulo chatbot
✓ Módulo vector_db
✗ .env file
✓ questions.csv

⚠ Hay problemas en la configuración que deben ser resueltos


In [3]:
# Importar el chatbot y sus componentes
from agents.agent_manager import AgentManager
from chatbot.core import CubaChatbot
from vector_db.chroma_storage import VectorStorage
from chatbot.core import CubaChatbot
from chatbot.gap_detector import GapDetector
from crawlers.dynamic_crawler import DynamicCrawler
from agents.retriever_agent import RetrieverAgent
from agents.generator_agent import GeneratorAgent
from agents.gap_detector_agent import GapDetectorAgent
from agents.updater_agent import UpdaterAgent
from agents.agent_manager import AgentManager
from agents.guide_agent import GuideAgent
from agents.planner_agent import TravelPlannerAgent

# Inicializar el chatbot
def initialize_chatbot():
    try:
        print("\nInicializando Chatbot...")
        chatbot = CubaChatbot()
        print("Chatbot inicializado correctamente")
        
        return chatbot
    except ImportError as e:
        print(f"\nError de importación: {str(e)}")
        print("Verifica que:")
        print("1. Estás en el directorio correcto")
        print("2. El directorio 'src' está en el Python path")
        print(f"3. Los módulos están disponibles en: {src_dir}")
        return None
    except Exception as e:
        print(f"\nError inesperado: {str(e)}")
        print(f"Tipo de error: {type(e).__name__}")
        import traceback
        print("\nDetalles del error:")
        print(traceback.format_exc())
        return None

# Inicializar el chatbot
print("=== Iniciando configuración del chatbot ===")
chatbot = initialize_chatbot()

if chatbot is not None:
    print("\n✓ Chatbot inicializado exitosamente")
    print(f"Tipo de chatbot: {type(chatbot).__name__}")
else:
    print("\n✗ Error al inicializar el chatbot")

=== Iniciando configuración del chatbot ===

Inicializando Chatbot...


c:\Users\Janet\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chatbot inicializado correctamente

✓ Chatbot inicializado exitosamente
Tipo de chatbot: CubaChatbot


In [4]:
# Cargar las preguntas desde el archivo CSV
def load_questions():
    try:
        questions_df = pd.read_csv('questions.csv')
        return questions_df['pregunta'].tolist()
    except Exception as e:
        print(f"Error al cargar las preguntas: {str(e)}")
        return []

# Cargar las preguntas
questions = load_questions()
print(f"Se cargaron {len(questions)} preguntas")

Se cargaron 30 preguntas


In [5]:
def generate_response(prompt):
    chatbot = CubaChatbot()
    if not chatbot.vector_db.get_documents():
        print("\nCargando datos iniciales...\n")
        try:
            chatbot.vector_db.reload_data()
            if not chatbot.vector_db.get_documents():
                print("Error: No se pudieron cargar los datos iniciales")
        except Exception as e:
            raise ValueError(f"Error crítico: {str(e)}")

    detector = GapDetector(chatbot.vector_db)
    updater = DynamicCrawler()
    guide_agent = GuideAgent(chatbot.vector_db)
    planner_agent = TravelPlannerAgent(chatbot.vector_db)

    # Inicialización de agentes
    retriever_agent = RetrieverAgent(chatbot.vector_db)
    generator_agent = GeneratorAgent(guide_agent, planner_agent)
    gap_detector_agent = GapDetectorAgent(detector)
    updater_agent = UpdaterAgent(updater)

    # Inicialización del manager
    manager = AgentManager([
        retriever_agent,
        generator_agent,
        gap_detector_agent,
        updater_agent
    ])

    # Procesar input del usuario
    # Recuperar contexto (opcional, si lo usas)
    retrieval_task = {"type": "retrieve", "query": prompt}
    context = manager.dispatch(retrieval_task, {})

    # Generar respuesta
    generate_task = {"type": "generate", "prompt": prompt}
    response = manager.dispatch(generate_task, context)

    # Verificar si necesita actualización
    detect_task = {"type": "detect_gap", "prompt": prompt, "response": response}
    needs_update = manager.dispatch(detect_task, context)

    if needs_update:
        # Identificar fuentes a actualizar
        sources = detector.identify_outdated_sources(prompt)
        update_task = {"type": "update_sources", "sources": sources}
        manager.dispatch(update_task, context)
        chatbot.vector_db.reload_data()
        # Regenerar respuesta
        response = manager.dispatch(generate_task, context)

    # Agregar respuesta final
    if hasattr(response, "choices"):
        response_text = " ".join([choice.message.content for choice in response.choices])
    else:
        response_text = str(response)

    return response_text

In [6]:
# Función para procesar las preguntas y obtener respuestas
def process_questions(questions):
    results = []
    
    for i, question in enumerate(questions, 1):
        try:
            print(f"\nProcesando pregunta {i}/{len(questions)}")
            print(f"Pregunta: {question}")
            
            # Obtener respuesta del chatbot
            print("Enviando pregunta al chatbot...")
            response = generate_response(question)
            print("Respuesta recibida del chatbot")
            
            # Procesar la respuesta según su tipo
            if response is None:
                raise ValueError("La respuesta del chatbot es None")
                
            if isinstance(response, dict):
                if 'response' in response:
                    response_text = response['response']
                elif 'message' in response:
                    response_text = response['message']
                else:
                    print(f"Estructura de respuesta inesperada: {response.keys()}")
                    response_text = str(response)
            else:
                response_text = str(response)
            
            print(f"Longitud de la respuesta: {len(response_text)} caracteres")
            
            # Guardar resultado
            results.append({
                'pregunta': question,
                'respuesta_manual': response_text
            })
                
        except Exception as e:
            print(f"Error procesando pregunta {i}: {str(e)}")
            results.append({
                'pregunta': question,
                'respuesta': f"ERROR: {str(e)}",
                'id': i
            })
    
    return results

In [7]:
# Ejecutar el experimento
if not questions:
    print("Error: No se pudieron cargar las preguntas")
    print("Verifica que:")
    print("1. El archivo 'questions.csv' existe en el directorio actual")
    print("2. El archivo contiene una columna llamada 'pregunta'")
else:
    print(f"Chatbot inicializado: {type(chatbot).__name__}")
    print(f"Preguntas cargadas: {len(questions)}")
    print("\nIniciando procesamiento de preguntas...")
    
    results = process_questions(questions)
    
# Guardar resultados finales en JSON
results_json = [
    {
        "pregunta": r["pregunta"],
        "respuesta": r.get("respuesta_manual", "")
    } 
    for r in results
]

with open('responses.json', 'w', encoding='utf-8') as f:
    json.dump(results_json, f, ensure_ascii=False, indent=2)

print("\nProceso completado. Resultados guardados en 'responses.json'")

# Mostrar algunas estadísticas
print(f"\nEstadísticas:")
print(f"Total de preguntas procesadas: {len(results)}")
print(f"Preguntas con error: {len([r for r in results if 'ERROR' in str(r.get('respuesta', ''))])}")

# Verificar formato para comparison.ipynb
required_keys = {'pregunta', 'respuesta'}
if all(key in results[0] for key in required_keys):
    print("\nFormato correcto para comparison.ipynb ✓")
else:
    print("\nAdvertencia: El formato puede no ser compatible con comparison.ipynb")
    print(f"Claves esperadas: {required_keys}")
    print(f"Claves actuales en el primer registro: {set(results[0].keys())}")

Chatbot inicializado: CubaChatbot
Preguntas cargadas: 30

Iniciando procesamiento de preguntas...

Procesando pregunta 1/30
Pregunta: ¿Qué opciones de alojamiento hay en Cienfuegos?
Enviando pregunta al chatbot...


2025-06-14 14:21:56.748 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 14:21:56.749 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


Creando nueva colección: Collection [cuba_tourism_data] does not exists


Unhandled error in Deferred:

Traceback (most recent call last):
  File "c:\Users\Janet\AppData\Local\Programs\Python\Python312\Lib\site-packages\twisted\internet\defer.py", line 1857, in _inlineCallbacks
    result = context.run(gen.send, result)
  File "c:\Users\Janet\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\crawler.py", line 154, in crawl
    self._apply_settings()
  File "c:\Users\Janet\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\crawler.py", line 124, in _apply_settings
    verify_installed_reactor(reactor_class)
  File "c:\Users\Janet\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\utils\reactor.py", line 171, in verify_installed_reactor
    raise RuntimeError(
builtins.RuntimeError: The installed reactor (twisted.internet.selectreactor.SelectReactor) does not match the requested one (twisted.internet.asyncioreactor.AsyncioSelectorReactor)

2025-06-14 14:22:45.395 WARNING streamlit.runtime.scriptrunner_utils.script_run

Se cargaron 198 documentos


...

Respuesta recibida del chatbot
Longitud de la respuesta: 741 caracteres

Procesando pregunta 29/30
Pregunta: ¿Dónde puedo encontrar hoteles con vistas al mar en La Habana?
Enviando pregunta al chatbot...


2025-06-14 14:33:30.798 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Respuesta recibida del chatbot
Longitud de la respuesta: 771 caracteres

Procesando pregunta 30/30
Pregunta: ¿Qué lugares históricos son destacados en Santiago de Cuba?
Enviando pregunta al chatbot...


2025-06-14 14:33:43.617 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Respuesta recibida del chatbot
Longitud de la respuesta: 792 caracteres

Proceso completado. Resultados guardados en 'responses.json'

Estadísticas:
Total de preguntas procesadas: 30
Preguntas con error: 0

Advertencia: El formato puede no ser compatible con comparison.ipynb
Claves esperadas: {'respuesta', 'pregunta'}
Claves actuales en el primer registro: {'respuesta_manual', 'pregunta'}
